In [ ]:
#@title Required Installations
!pip install swig
!pip install gymnasium
!pip install mujoco
!pip install gymnasium[mujoco]
!pip install stable-baselines3[extra]

In [ ]:
!pip install tensorboard
!pip install jupyter-tensorboard

  Preparing metadata (setup.py) ... done
  Created wheel for jupyter-tensorboard: filename=jupyter_tensorboard-0.2.0-py2.py3-none-any.whl size=15244 sha256=8c2a53a6a0ff4d430194bd9b47e036d41aa9fd67081fdb6b973290b293694603
  Stored in directory: /root/.cache/pip/wheels/6a/e8/ee/64103cb21821f6a747a19b93a417b7e925857daab8ec5c7f50
Successfully built jupyter-tensorboard


In [ ]:
#@title Check if installation was successful

from google.colab import files

import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl
Checking that the installation succeeded:
Installation successful.


In [ ]:
import gymnasium as gym
import stable_baselines3
import mujoco

env = gym.make('CartPole-v1', render_mode="rgb_array") # checck thhis

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [ ]:
#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

Installing mediapy:


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
env = gym.make('CartPole-v1', render_mode="rgb_array") # checck thhis

In [ ]:
observation, info = env.reset(seed=42)

duaration = 5 # time - seconds
framerate = 60 # FPS

frames = [] # Empty list to append frames to

for _ in range(1000):
   action = env.action_space.sample()  # this is where you would insert your policy
   observation, reward, terminated, truncated, info = env.step(action)
   px = env.render()
   frames.append(px)

  #  media.show_image(env.render())
   if terminated or truncated:
      observation, info = env.reset()

env.close()

media.show_video(frames, fps=60)

This browser does not support the video tag.


In [ ]:
env = gym.make('CartPole-v1', render_mode="rgb_array") # checck thhis

In [ ]:
from stable_baselines3 import SAC, TD3, A2C
import os
import argparse

In [ ]:
model_dir = "models"
log_dir = "logs"
os.makedirs(model_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [ ]:
def train(env, a2c_algo):
    match a2c_algo:
        case 'SAC':
            model = SAC('MlpPolicy', env, verbose=1, device='cuda', tensorboard_log=log_dir)
        case 'TD3':
            model = TD3('MlpPolicy', env, verbose=1, device='cuda', tensorboard_log=log_dir)
        case 'A2C':
            model = A2C('MlpPolicy', env, verbose=1, device='cuda', tensorboard_log=log_dir)
        case _:
            print('Algorithm not found')
            return

    TIMESTEPS = 25000
    iters = 0
    while True:
        iters += 1

        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False)
        model.save(f"{model_dir}/{a2c_algo}_{TIMESTEPS*iters}")

In [ ]:
train(env,'A2C')

Streaming output truncated to the last 5000 lines.
|    ep_rew_mean        | 9.33      |
| time/                 |           |
|    fps                | 443       |
|    iterations         | 300       |
|    time_elapsed       | 3         |
|    total_timesteps    | 901500    |
| train/                |           |
|    entropy_loss       | -2.08e-08 |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 180299    |
|    policy_loss        | -0        |
|    value_loss         | 8.65      |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 9.25      |
|    ep_rew_mean        | 9.25      |
| time/                 |           |
|    fps                | 440       |
|    iterations         | 400       |
|    time_elapsed       | 4         |
|    total_timesteps    | 902000    |
| train/                |           |
|    entropy_loss       | -1.26e-20 |

KeyboardInterrupt: ignored

ls: cannot access '/logs': No such file or directory


In [ ]:
import os

log_dir = "/content/logs"  # Replace with the actual path to your log directory

# List the contents of the directory
directory_contents = os.listdir(log_dir)

# Display the permissions and ownership of the directory
for item in directory_contents:
    item_path = os.path.join(log_dir, item)
    item_info = os.stat(item_path)
    print(f"{item}: Permissions {oct(item_info.st_mode)[-4:]} | Owner: {item_info.st_uid} | Group: {item_info.st_gid}")


A2C_0: Permissions 0755 | Owner: 0 | Group: 0


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<IPython.core.display.Javascript object>

In [ ]:
def test(env, sb3_algo, path_to_model):

    match sb3_algo:
        case 'SAC':
            model = SAC.load(path_to_model, env=env)
        case 'TD3':
            model = TD3.load(path_to_model, env=env)
        case 'A2C':
            model = A2C.load(path_to_model, env=env)
        case _:
            print('Algorithm not found')
            return

    observation,info = env.reset(seed=42)
    done = False
    duaration = 5 # time - seconds
    framerate = 60 # FPS
    #extra_steps = 500
    frames = [] # Empty list to append frames to
    for _ in range(1000):
        action, _ = model.predict(observation)
        observation, reward, terminated, truncated, info= env.step(action)
        px = env.render()
        frames.append(px)
        if terminated or truncated:
            observation, info = env.reset()
    env.close()

    media.show_video(frames, fps=60)

In [ ]:
test(env,'A2C','models/A2C_1000000')

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


This browser does not support the video tag.
